# A simple linear baseline for the Walmart challenge
This notebook shows how you load the data, prepare it for usage with Keras and then create a submission file. The model is a simple linear regression.

In [ ]:
import pandas as pd
import numpy as np

## Loading the data
In Kaggle, data that can be accessed by a Kernel is saved under ``../inputs/``
From there we can load it with pandas:

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

We are going to do some data preparation. It is easiest to do this for training and test set combined so we have to do all these steps only once. It is good to know where to split the set afterwards though!

In [ ]:
len(train) # Get number of training examples

In [ ]:
len(test) # Get number of test examples

In [ ]:
df = pd.concat([train,test],axis=0) # Join train and test
a1 = df.iloc[:282451]
a2 = df.iloc[282451:]
w = len(df)
p = [1]*282451
p = np.array(p)
q = np.arange(282451)
a1 = a1.assign(p=p)
p = [0]*(w-282451)
p = np.array(p)
q = np.arange(w-282451)
a2 = a2.assign(p = p)
df = pd.concat([a1,a2], axis= 0)
q = np.arange(len(df))
df = df.assign(q=q)

In [ ]:
df.iloc[282451:]

In [ ]:
df.head() # Get an overview of the data

In [ ]:
df.describe()

There seem to be some missing values in the data. We have to make sure to deal with them before feeding anything into the network.

In [ ]:
df.isnull().sum()

We will do a bit of very basic feature engineering here by creating a feature which indicates whether a certain markdown was active at all.

In [ ]:
df = df.assign(md1_present = df.MarkDown1.notnull())
df = df.assign(md2_present = df.MarkDown2.notnull())
df = df.assign(md3_present = df.MarkDown3.notnull())
df = df.assign(md4_present = df.MarkDown4.notnull())
df = df.assign(md5_present = df.MarkDown5.notnull())

In [ ]:
df.isnull().sum()

We can probably safely fill all missing values with zero. For the markdowns this means that there was no markdown. For the weekly sales, the missing values are the ones we have to predict, so it does not really matter what we fill in there.

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
C = df.sort_values(by = ['Store', 'Dept', 'Date'])
z = C['Weekly_Sales'].values
Prev = [df['Weekly_Sales'].mean()]
for i in range(1,len(z)):
    Prev.append((z[i-1]))
for j in range(len(Prev)):
    if Prev[j] == 0:
        Prev[j] = Prev[j-1]
C = C.assign(Prev = np.array(Prev))
df = C

Now we have to create some dummy variebles for categorical data.

In [ ]:
# Make sure we can later recognize what a dummy once belonged to
df['Type'] = 'Type_' + df['Type'].map(str)
df['Store'] = 'Store_' + df['Store'].map(str)
df['Dept'] = 'Dept_' + df['Dept'].map(str)

In [ ]:
# Create dummies
type_dummies = pd.get_dummies(df['Type'])
store_dummies = pd.get_dummies(df['Store'])
dept_dummies = pd.get_dummies(df['Dept'])

In [ ]:
# Add dummies
df = pd.concat([df,type_dummies,store_dummies,dept_dummies],axis=1)

In [ ]:
# Remove originals
del df['Type']
del df['Store']
del df['Dept']

In [ ]:
del df['Date']
del df['CPI']
del df['Temperature']

In [ ]:
df.dtypes

Now we can split train test again.

In [ ]:
train = df[df['p'] == 1]
test = df[df['p'] == 0]
del train['p']
del test['p']
train = train.sort_values(by = ['q'])
test = test.sort_values(by = ['q'])
del train['q']
del test['q']

In [ ]:
test = test.drop('Weekly_Sales',axis=1) # We should remove the nonsense values from test

To get numpy arrays out of the pandas data frame, we can ask for a columns, or dataframes values

In [ ]:
y = train['Weekly_Sales'].values

In [ ]:
X = train.drop('Weekly_Sales',axis=1).values

In [ ]:
X.shape

Now we create the baseline model

In [ ]:
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras import regularizers

In [ ]:
model = Sequential()
model.add(Dense(1,input_dim=144,activation ='relu',kernel_regularizer= regularizers.l2(0.01)))
model.compile(optimizer='adam', loss='mae')

In [ ]:
y[10]

In [ ]:
model.fit(X,y,batch_size=2048,epochs=20)

After we have created our model, we can predict things with it on the test set

In [ ]:
model.evaluate(X,y)

In [ ]:
X_test = test.values

In [ ]:
test.head()

In [ ]:
y_pred = model.predict(X_test,batch_size=2048)

In [ ]:
y_pred[:10]

To create the ids required for the submission we need the original test file one more time

In [ ]:
testfile = pd.read_csv('../input/test.csv')

Now we create the submission. Once you run the kernel you can download the submission from its outputs and upload it to the Kaggle InClass competition page.

In [ ]:
submission = pd.DataFrame({'id':testfile['Store'].map(str) + '_' + testfile['Dept'].map(str) + '_' + testfile['Date'].map(str),
                          'Weekly_Sales':y_pred.flatten()})

In [ ]:
submission.to_csv('submission.csv',index=False)